In [0]:
import gspread 
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
from oauth2client.service_account import ServiceAccountCredentials
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from mpl_toolkits.mplot3d import Axes3D
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 300)

In [0]:
%matplotlib inline

In [0]:
KEY_FILE_LOCATION = '#'
scope = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/spreadsheets','https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
            KEY_FILE_LOCATION, scopes=scope)
gc = gspread.authorize(credentials)

In [0]:
doc = '#'
workbook_read   = gc.open(doc)#読み書きするgoogle spreadsheet
duration = '20200201-20200229'
data_type_1 = 'review_click_3brands'
data_type_2 = 'call_tracking'
worksheet_read_1 = workbook_read.worksheet('raw' + '_'+ data_type_1 +'_'+ duration)
worksheet_read_2 = workbook_read.worksheet('raw' + '_' + data_type_2 + '_' + duration)
rows_1 = worksheet_read_1.get_all_values()
rows_2 = worksheet_read_2.get_all_values()
columns = {'ブランド名':'Brand','店舗名':'Shop','クリック数':'Review','クリック率(%)':'%_Review','全件':'Call','通話':'Call_through','通話中':'Call_not_through','キャンセル':'Cancel'}

In [0]:
#データ読み込み
Review = pd.DataFrame(rows_1[:][1: len(rows_1)], columns=rows_1[0]).reset_index().drop('index', axis=1).rename(columns=columns)
Call = pd.DataFrame(rows_2[:][1: len(rows_2)], columns=rows_2[0]).reset_index().drop('index', axis=1).rename(columns=columns)

In [0]:
Review_Call = pd.merge(Review, Call, how='inner', left_on=['Brand','Shop'],right_on=['Brand','Shop']).drop('期間',axis=1).drop('計測月',axis=1).set_index(['Brand', 'Shop']).astype(float).rename({'VANSAN':'Vansan','串カツ田中':'Tanaka','すみれ':'Sumire'},axis=0).reset_index()

In [0]:
Sumire = Review_Call[Review_Call['Brand'] == 'Sumire']
Tanaka = Review_Call[Review_Call['Brand'] == 'Tanaka']
Vansan = Review_Call[Review_Call['Brand'] == 'Vansan']
Sumire_Review = Sumire.loc[:,'Review']
Sumire_Call = Sumire.loc[:,'Call']
Tanaka_Review = Tanaka.loc[:,'Review']
Tanaka_Call = Tanaka.loc[:,'Call']
Vansan_Review = Vansan.loc[:,'Review']
Vansan_Call = Vansan.loc[:,'Call']

In [0]:
fig = plt.figure(figsize=(12,8))
plt.xticks(np.arange(0, 150, 25))
plt.yticks(np.arange(0, 350, 50))

plt.scatter(np.array(Sumire_Review), np.array(Sumire_Call), s=40, c='red', label='Sumire')
plt.scatter(np.array(Tanaka_Review), np.array(Tanaka_Call), s=40, c='blue', label='Tanaka')
plt.scatter(np.array(Vansan_Review), np.array(Vansan_Call), s=40, c='black', label='Vansan')
plt.xlabel('The number of Reviews')
# add a label to the y axis
plt.ylabel('The number of Calls')
# add a title
plt.title('Scatter plot between Reviews and Calls')
# add a legend (uses the labels from plt.scatter)
plt.legend()
# add the legend to loc=4 (the lower right hand corner), also gets rid of the frame and adds a title
plt.legend(loc=1, frameon=False, title='Legend')
plt.rcParams["font.size"] = 20
plt.tight_layout()
fig.savefig('Scatter_plot_between_Reviews_and_Calls_before_normalize.png')

In [0]:
plt.figure(figsize=(12,8))
plt.xticks(np.arange(-4, 4, 1))
plt.yticks(np.arange(-4, 4, 1))

plt.scatter((np.array(Sumire_Review)-Sumire_Review.mean())/Sumire_Review.std(), (np.array(Sumire_Call)-Sumire_Call.mean())/Sumire_Call.std(), s=40, c='red', label='Sumire')
plt.scatter((np.array(Tanaka_Review)-Tanaka_Review.mean())/Tanaka_Review.std(), (np.array(Tanaka_Call)-Tanaka_Call.mean())/Tanaka_Call.std(), s=40, c='blue', label='Tanaka')
plt.scatter((np.array(Vansan_Review)-Vansan_Review.mean())/Vansan_Review.std(), (np.array(Vansan_Call)-Vansan_Call.mean())/Vansan_Call.std(), s=40, c='black', label='Vansan')
plt.xlabel('The number of Reviews')
# add a label to the y axis
plt.ylabel('The number of Calls')
# add a title
plt.title('Scatter plot between Reviews and Calls')
# add a legend (uses the labels from plt.scatter)
plt.legend()
# add the legend to loc=4 (the lower right hand corner), also gets rid of the frame and adds a title
plt.legend(loc=1, frameon=False, title='Legend')
plt.rcParams["font.size"] = 20
plt.tight_layout()
plt.savefig('Scatter_plot_between_Reviews_and_Calls_afer_normalize.png')

In [0]:
import seaborn as sns
from scipy import stats
plt.subplot?
sns.set_style('white')

In [0]:
plt.figure(figsize=(12,8))
sns.jointplot(x='Review', y='Call', data=Review_Call, kind='reg', height=10).annotate(stats.pearsonr);
plt.title('Joinplot between Reviews and Calls for all',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_all.png')

In [0]:
sns.jointplot(x='Review', y='Call', data=Sumire, kind='reg', height=10).annotate(stats.pearsonr);
plt.title('Joinplot between Reviews and Calls for Sumire',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_Sumire.png')

In [0]:
sns.jointplot(x='Review', y='Call', data=Tanaka, kind='reg',height=10).annotate(stats.pearsonr);
plt.title('Joinplot between Reviews and Calls for Tanaka',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_Tanaka.png')

In [0]:
sns.jointplot(x='Review', y='Call', data=Vansan, kind='reg', height=10).annotate(stats.pearsonr)
plt.title('Joinplot between Reviews and Calls for VANSAN',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_VANSAN.png')

In [0]:
sns.lmplot(x='Review', y='Call', hue='Brand',data=Review_Call, height=10, palette="Set1")
plt.savefig('Lm_plot_between_Reviews_and_Calls_for_all.png')

In [0]:
sns.lmplot(x='Review', y='Call', hue='Brand',data=Review_Call, height=10, palette="Set1", col="Brand", aspect=.4, x_jitter=.1)
plt.savefig('Lm_lmplot_between_Reviews_and_Calls_for_all_separately.png')

In [0]:
#レビュー点数の高さもコール数へ影響？

In [0]:
#メタデータ
data_type_3 = 'score_review'
worksheet_read_3 = workbook_read.worksheet('raw' + '_' + data_type_3 + '_' + duration)
columns = {'ブランド名':'Brand','店舗名':'Shop','クリック数':'Review','クリック率(%)':'%_Review','全件':'Call','通話':'Call_through','通話中':'Call_not_through','キャンセル':'Cancel','Sc"ブランド"':'Brand','店舗名':'Shop','スコア':'review_score','合計レビュー数':'review_count'}
#データ読み込み
rows_3 = worksheet_read_3.get_all_values()
Review = pd.DataFrame(rows_1[:][1: len(rows_1)], columns=rows_1[0]).reset_index().drop('index', axis=1).rename(columns=columns)
Call = pd.DataFrame(rows_2[:][1: len(rows_2)], columns=rows_2[0]).reset_index().drop('index', axis=1).rename(columns=columns)
Score_R =  pd.DataFrame(rows_3[:][1: len(rows_3)], columns=rows_3[0]).reset_index().drop('index', axis=1).rename(columns=columns)

In [0]:
Review_Call = pd.merge(Review, Call, how='inner', left_on=['Brand','Shop'],right_on=['Brand','Shop']).drop('期間',axis=1).drop('計測月',axis=1)
Review_Call

In [0]:
Review_Call_Score = pd.merge(Review_Call, Score_R, how='inner', left_on=['Brand','Shop'], right_on=['Brand','Shop']).drop('import_at',axis=1).set_index(['Brand', 'Shop']).astype(float).rename({'VANSAN':'Vansan','串カツ田中':'Tanaka','すみれ':'Sumire'},axis=0).reset_index()
Review_Call_Score

In [0]:
Sumire = Review_Call_Score[Review_Call_Score['Brand'] == 'Sumire']
Tanaka = Review_Call_Score[Review_Call_Score['Brand'] == 'Tanaka']
Vansan = Review_Call_Score[Review_Call_Score['Brand'] == 'Vansan']
Sumire_Review = Sumire.loc[:,'Review']
Sumire_Call = Sumire.loc[:,'Call']
Sumire_Score = Sumire.loc[:,'review_score']
Tanaka_Review = Tanaka.loc[:,'Review']
Tanaka_Call = Tanaka.loc[:,'Call']
Tanaka_Score = Tanaka.loc[:,'review_score']
Vansan_Review = Vansan.loc[:,'Review']
Vansan_Call = Vansan.loc[:,'Call']
Vansan_Score = Tanaka.loc[:,'review_score']

In [0]:
len(Tanaka),len(Sumire),len(Vansan)

In [0]:
plt.figure()
sns.pairplot(Review_Call_Score.loc[:,['Brand','Review','Call','review_score']], hue='Brand', diag_kind='kde', size=5);
plt.savefig('Pair_plot_between_Reviews_Calls_and_review_score_for_all.png')

In [0]:
Review_Call_Score['Review_std'] = preprocessing.scale(Review_Call_Score.loc[:,['Review']])
Review_Call_Score['Call_std'] = preprocessing.scale(Review_Call_Score.loc[:,['Call']])
Review_Call_Score['review_score_std'] = preprocessing.scale(Review_Call_Score.loc[:,['review_score']])
Review_Call_Score

In [0]:
Review_Call_Score.loc[:,['Review','Call','review_score']].corr()

In [0]:
Sumire.loc[:,['Review','Call','review_score']].corr()

In [0]:
Tanaka.loc[:,['Review','Call','review_score']].corr()

In [0]:
Vansan.loc[:,['Review','Call','review_score']].corr()

In [0]:
sns.jointplot(x='review_score', y='Call', data=Review_Call_Score, kind='reg', height=10).annotate(stats.pearsonr)
plt.title('Joinplot between Review Score and Calls for all',fontsize=12)
plt.savefig('Join_plot_between_review_score_and_Calls_for_all.png')

In [0]:
sns.jointplot(x='review_score', y='Call', data=Tanaka, kind='reg', height=10).annotate(stats.pearsonr)
plt.title('Joinplot between Review Score and Calls for Tanaka',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_Tanaka.png')

In [0]:
plt.figure(figsize=(10,10))
sns.jointplot(x='review_score', y='Call', data=Sumire, kind='reg', height=10).annotate(stats.pearsonr)
plt.title('Joinplot between Review Score and Calls for Sumire',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_Sumire.png')

In [0]:
plt.figure(figsize=(10,10))
sns.jointplot(x='review_score', y='Call', data=Vansan, kind='reg', height=10).annotate(stats.pearsonr)
plt.title('Joinplot between Review Score and Calls for VANSAN',fontsize=12)
plt.savefig('Join_plot_between_Reviews_and_Calls_for_VANSAN.png')

In [0]:
sns.lmplot(x='review_score', y='Call', hue='Brand',data=Review_Call_Score, height=10, palette="Set1")
plt.title('Lmplot between Review Score and Calls for all',fontsize=12)
plt.savefig('Lm_plot_between_Review_Score_and_Calls_for_all.png')

In [0]:
sns.lmplot(x='review_score', y='Call', hue='Brand',data=Review_Call_Score, height=10, palette="Set1", col="Brand", aspect=.4, x_jitter=.1)
plt.title('Lmplot between Review Score and Calls for all seaparately',fontsize=12)
plt.savefig('Lm_lmplot_between_Review_Score_and_Calls_for_all_separately.png')

In [0]:
LinearModel = LinearRegression()

In [0]:
model_lr = LinearRegression()
X = Review_Call_Score.loc[:,['Review','review_score']]
Y = Review_Call_Score.loc[:,['Call']]
model_lr.fit(X,Y)
# 回帰係数
print(model_lr.coef_)
# 切片 (誤差)
print(model_lr.intercept_)
# 決定係数
print(model_lr.score(X, Y))

In [0]:
plt.figure(figsize=(12,8))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
 
x = Review_Call_Score.loc[:,['Review']]
y = Review_Call_Score.loc[:,['review_score']]
z = Review_Call_Score.loc[:,['Call']]
 
ax.scatter(x, y, z)

In [0]:
X = Review_Call_Score.loc[:,['Review','review_score']]
X = sm.add_constant(X)
Y = Review_Call_Score.loc[:,['Call']]
# 最小二乗法でモデル化
model = sm.OLS(Y, X)
result = model.fit()

# 重回帰分析の結果を表示する
result.summary()